https://python.langchain.com/docs/use_cases/sql/quickstart/

In [21]:
from openai import OpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.chat_models import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

from pyprojroot import here
import pandas as pd 
import re
import warnings
warnings.filterwarnings("ignore")

Load csv data to data frame

In [23]:
df = pd.read_csv(here("/Users/ashokarulsamy/Python Learning/itam/data/itam_files/itam_hw.csv"))
print(df.shape)
print(df.columns.tolist())
display(df.head(3))

(100, 9)
['Asset_ID', 'Asset_Name', 'Asset_Type', 'Manufacturer', 'Model', 'Purchase_Date', 'Warranty_Expiry', 'Assigned_To', 'Status']


,Asset_ID,Asset_Name,Asset_Type,Manufacturer,Model,Purchase_Date,Warranty_Expiry,Assigned_To,Status
0,HW001,It Device,Hardware,Lenovo,Model-4915,2022-07-07,2027-12-29,William Jackson,Retired
1,SW002,Source Software,Software,SAP,Version-82.2,2022-11-06,2027-04-17,Ashok Arulsamy,Active
2,HW003,Green Device,Hardware,HP,Model-8953,2020-09-28,2027-03-16,Jennifer Adams,Retired


Load data from data frame to database

In [ ]:
db_path = "/Users/ashokarulsamy/Python Learning/itam/data/sqlite_db/sqlite_itam.db"
db_path = f"sqlite:///{db_path}"
engine = create_engine(db_path)
#df.to_sql("itam_hw", engine, index=False)

Create DB object

In [11]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM itam_hw where assigned_to = 'Ashok Arulsamy'")

sqlite
['itam_hw']


"[('SW002', 'Source Software', 'Software', 'SAP', 'Version-82.2', '2022-11-06', '2027-04-17', 'Ashok Arulsamy', 'Active'), ('HW013', 'Majority Device', 'Hardware', 'Lenovo', 'Model-4924', '2022-04-22', '2029-10-02', 'Ashok Arulsamy', 'In Use'), ('HW024', 'Card Device', 'Hardware', 'Lenovo', 'Model-0757', '2022-12-27', '2028-05-04', 'Ashok Arulsamy', 'Retired'), ('HW053', 'Step Device', 'Hardware', 'Asus', 'Model-6226', '2022-10-14', '2029-05-23', 'Ashok Arulsamy', 'Available')]"

Create LLM object

In [13]:
llm = ChatOpenAI(
    api_key="",
    model_name="gpt-4o-mini",
    temperature=0.0,
    max_tokens=1000
)

Convert user query into corresponding SQL query

In [14]:
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many hardware assets are there"})
print(response)

SQLQuery: SELECT COUNT(*) AS "Hardware_Asset_Count" FROM itam_hw WHERE "Asset_Type" = 'Hardware';


Utility function to clean up the sql query

In [18]:
def parse_output(output):
    if response.startswith("SQLQuery: "):
        executable_query = response.replace("SQLQuery: ", "", 1)  # Remove first occurrence
        #print(executable_query)
    
    if executable_query.startswith("SQLQuery: "):
        executable_query = executable_query.replace("SQLQuery: ", "", 1)  # Remove any more occurrence
        #print(executable_query)
    
    matches = re.findall(r"```sql\n(.*?)```", response, re.DOTALL)

    if matches:
        executable_query = matches[0].strip()  # Get the first match and clean up whitespace
        #print("Executable SQL Query:")
        #print(executable_query)
    return executable_query.strip()

Clenup up the SQL query

In [19]:
clean_query = parse_output(response)
clean_query

'SELECT COUNT(*) AS "Hardware_Asset_Count" FROM itam_hw WHERE "Asset_Type" = \'Hardware\';'

Run the query to validate

In [20]:
db.run(clean_query)

'[(56,)]'

All in one (Convert user query into sql query and execute)

In [22]:
write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | parse_output | execute_query

chain.invoke({"question": "How many hardware assets are there"})


'[(56,)]'